In [6]:
import soundfile as sf
from transformers import WhisperFeatureExtractor,AutoTokenizer
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
import torch
device = "cuda" if torch.cuda.is_available() else "CPU"

In [9]:
from transformers import WhisperForConditionalGeneration
my_feature_extractor = WhisperFeatureExtractor.from_pretrained(f"output/whisper-large-v3-ar_v1")
my_tokenizer = WhisperTokenizer.from_pretrained(f"output/whisper-large-v3-ar_v1", language="arabic", task="transcribe")
my_processor = WhisperProcessor.from_pretrained(f"output/whisper-large-v3-ar_v1", language="arabic", task="transcribe")
my_model = WhisperForConditionalGeneration.from_pretrained(f"output/whisper-large-v3-ar_v1/checkpoint-39000")
my_model = my_model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [10]:
import librosa
def split_audio(audio_path, chunk_duration=30):
    audio, sr = librosa.load(audio_path, sr=16000)
    chunks = []
    for i in range(0, len(audio), chunk_duration * sr):
        chunks.append(audio[i:i + chunk_duration * sr])
    return chunks

In [29]:
chunks = split_audio("test samples/AUDIO-2024-12-10-22-16-08.wav", chunk_duration=30)

In [30]:
input_features = my_feature_extractor(chunks, sampling_rate=my_feature_extractor.sampling_rate).input_features
input_features = {"input_features": input_features}
input_features = my_processor.feature_extractor.pad(input_features, return_tensors="pt")
input_features = input_features.to(device)

In [31]:
my_model.eval()
my_model.generation_config.language = "arabic"
my_model.generation_config.task = "transcribe"
my_model.generation_config.forced_decoder_ids = None

In [32]:
len(chunks)

6

In [33]:
import time
start_time = time.time()
my_pred = my_model.generate(input_features['input_features'],)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.6f} seconds")

Execution Time: 41.396215 seconds


In [34]:
my_transcription = my_processor.batch_decode(my_pred.view(1,-1), skip_special_tokens=True,)

In [38]:
ground_truth = "شكرا لاتصالكم بمفروشات الرقيب معاك خالد كيف أقدر أخدمك؟ ألو هلا وغلا كيف أقدر أساعدك اليوم؟ لو سمحت أقدر أكنت أنا دحين اشتريت حاجات و أقدر أرجعها ولا لأ؟ اذا ما استلمت الحاجات تقدر تلغي الطلب خلال سبعة أيام من تاريخ الشراء وتسترجع المبلغ كامل، أما إذا استلمتها تقدر تستبدلها خلال أربعطعش يوم من تاريخ الشراء بشرط تكون غير مستخدمة وبحالتها الأصلية، إذا تحتاج مساعدة إضافية خبرني طيب آآ أنا لسه ما استلمتها يعني في آآ كنبة اشتريتها أبى أبدلها في كومودينات ينفع ولا لأ؟إذا لسه ما استلمت الكنبة تقدر تلغي الطلب وتسترجع المبلغ كامل أو تستبدلها بالكوميدينات إللي تبيها تواصل مع خدمة العملاء عشان يرتبون لك العملية تحتاج رقم التواصل إيوة الله يسعدك يا ريت الله يسعدك تفضل الرقم تسعة إثنين صفر صفر صفر إثنين خمسة أربعة صفرالفريق هناك راح يساعدك في ترتيب عملية التبديل أي خدمة ثانية طب أنا الحين متصلة على نفس هذا الرقم اللي ديتني هو صحيح هذا هو نفس الرقم اللي تتواصلين عليه حاليا إذا عندك أي استفسار أو تحتاجين مساعدة إضافية أنا هنا لخدمتك تقدر تحولني خدمة العملاء عشان أتكلم معاهمأعتذر منك ما أقدر أحول المكالمة مباشرة لكن تواصلي معهم بنفس الرقم في أوقات العمل وإن شاء الله راح يساعدونك بكل اللي تحتاجينه إذا في أي شيء ثاني أقدر متى أوقات العمل لو سمحت أوقات العمل لقسم خدمة العملاء من السبت إلى الخميس من الساعة تسعة صباحا إلى الساعة الحادية عشر مساء ويوم الجمعة من الساعة أربعة عصرا إلى"

In [36]:
print("My trained model prediction:")
print(my_transcription[0])

My trained model prediction:
شكرا لاتصالكم بمفروشات الرقيب معاك خالد كيف أقدر أخدمك؟ ألو هلا وغلا كيف أقدر أساعدك اليوم؟ لو سمحت أقدر أكنت أنا دحين اشتريت حاجات و أقدر أرجعها ولا لأ؟تقدر تلغي الطلب خلال سبعة أيام من تاريخ الشراء وتسترجع المبلغ كامل، أما إذا استلمتها تقدر تستبدلها خلال أربعطعش يوم من تاريخ الشراء بشرط تكون غير مستخدمة وبحالتها الأصلية، إذا تحتاج مساعدة إضافية خبرني طيب آآ أنا لسه ما استلمتها يعني في آآ كنبة اشتريتها أبى أبدلها في كومودينات ينفع ولا لأ؟إذا لسه ما استلمت الكنبة تقدر تلغي الطلب وتسترجع المبلغ كامل أو تستبدلها بالكوميدينات إللي تبيها تواصل مع خدمة العملاء عشان يرتبون لك العملية تحتاج رقم التواصل إي والله يسعدك يا ريت الله يسعدك تفضل الرقم تسعة إثنين صفر صفر صفر إثنين خمسة أربعة صفرالفريق هناك راح يساعدك في ترتيب عملية التبديل أي خدمة ثانية طب أنا الحين متصلة على نفس هذا الرقم اللي ديتني هو صحيح هذا هو نفس الرقم اللي تتواصلين عليه حاليا إذا عندك أي استفسار أو تحتاجين مساعدة إضافية أنا هنا لخدمتك تقدر تحولني خدمة العملاء عشان أتكلم معاهمأعتذر منك ما أقدر أحول

In [37]:
import evaluate

metric = evaluate.load("wer")

In [44]:
google_transcribe = """شكرا لاتصالكم بمفروشات الرْɠɑيب. معاك خالد، كيف أقدر أخّدِمك؟ الو.هلا وغلا! كيف أقدر أساعدك اليوم؟ لو سمحت اقدر اكنت انا دحين اشتريت حاجات واقدر ارجعها ولا لا؟ إذا ما استلمت الحاجات، تقدر تلغي الطلب خلال سبعة أيام من تاريخ الشراء وتسترجع المبلغ كامل. أما إذا استلمتها، تقدر تستبدلها خلال أربعطعش يوم من تاريخ الشراء بشرط تكون غير مستخدمة وبحالتها الأصلية. إذا تحتاج مساعدة إضافية، خبرني! طيب انا لسه ما استلمتها يعني في كنبه اشتريتها او ابدلها في كوميدينات ينفع ولا لا؟ إذا لسه ما استلمت الكنبة، تقدر تلغي الطلب وتسترجع المبلغ كامل، أو تستبدلها بالكوميدينات اللي تبيها. تواصل مع خدمة العملاء عشان يرتبون لك العملية. تحتاج رقم التواصل؟ اي والله يسعدك. الله يسعدك! تفضل الرقم: تسعة.. اثنين، صفر.. صفر، صفر، اثنين، خمسة، أربعة، صفر. الفريق هناك راح يساعدك في ترتيب عملية التبديل. أي خدمة ثانية؟ طب انا الحين متصله على نفس هذا الرقم اللي اديتني هون. صحيح، هذا هو نفس الرقم اللي تتواصلين عليه حالياً. إذا عندك أي استفسار أو تحتاجين مساعدة إضافية، أنا هنا لخدمتك! تقدر تحولني خدمه العملاء عشان اتكلم معاهم. أعتذر منك، ما أقدر أحول المكالمة مباشرة. لكن تواصلي معهم بنفس الرقم في أوقات العمل، وإن شاء الله راح يساعدونك بكل اللي تحتاجينه. إذا في أي... متى؟ متى اوقات العمل لو سمحت؟"""

In [46]:
print("My trained model WER (Word error rate):")
print(100 * metric.compute(predictions=my_transcription, references=[ground_truth]))
print("Deepgram model WER (Word error rate):")
print(100 * metric.compute(predictions=[google_transcribe], references=[ground_truth]))

My trained model WER (Word error rate):
4.25531914893617
Deepgram model WER (Word error rate):
42.97872340425532
